In [1]:
from dotenv import load_dotenv
load_dotenv("../.env.prod")

True

In [2]:
import pandas as pd

In [3]:
df = pd.read_json("../files/raw/cities_col.json")
print(len(df[df["location"].isna()]))

0


In [4]:
ciudades= df[df["location"].isna()][["name","department_name"]]

In [5]:
print(len(df[df["location"].isna()]))
df = df[~df["location"].isna()]

0


In [ ]:
import json
import numpy as np

EARTH_R_KM = 6371.0088

def latlon_to_unit_xyz(lat_deg, lon_deg):
    """Vectorized: lat_deg, lon_deg can be scalars, 1D arrays, or Nx1 arrays."""
    lat = np.radians(lat_deg)
    lon = np.radians(lon_deg)
    x = np.cos(lat) * np.cos(lon)
    y = np.cos(lat) * np.sin(lon)
    z = np.sin(lat)
    return np.stack([x, y, z], axis=-1)  # (..., 3)

class GeoFourierEncoder:
    """

    """
    def __init__(self, D=8, scales_km=(5000, 2000, 700, 250), seed=0):
        assert D % 2 == 0, "D must be even (we emit sin+cos pairs)."
        self.D = D
        self.m = D // 2
        self.scales_km = np.array(scales_km, dtype=float).ravel()
        if self.scales_km.size == 1:
            self.scales_km = np.repeat(self.scales_km, self.m)
        elif self.scales_km.size < self.m:
            # Log-space interpolate to fill to length m
            t = np.linspace(0, 1, self.m)
            tin = np.linspace(0, 1, self.scales_km.size)
            self.scales_km = np.exp(np.interp(t, tin, np.log(self.scales_km)))
        elif self.scales_km.size > self.m:
            self.scales_km = self.scales_km[:self.m]
        self.seed = seed
        self.B = self._build_B()

    def _build_B(self):
        rng = np.random.default_rng(self.seed)
        theta = self.scales_km / EARTH_R_KM                    
        ell_chord = 2.0 * np.sin(theta / 2.0)                 
        sigma = 1.0 / np.maximum(ell_chord, 1e-8)              # ω ~ N(0, 1/ell^2)
        B = rng.normal(loc=0.0, scale=sigma[:, None], size=(self.m, 3))
        return B.astype(np.float32)

    def transform(self, latlon_deg):
        """
        latlon_deg: array-like of shape (N, 2) in degrees (lat, lon).
        Returns: (N, D) features in [-1,1], L2-norm ~ O(1).
        """
        arr = np.asarray(latlon_deg, dtype=np.float32)
        xyz = latlon_to_unit_xyz(arr[:, 0], arr[:, 1])         
        proj = xyz @ self.B.T                                  
        feat = np.concatenate([np.cos(proj), np.sin(proj)], axis=-1)
        feat *= (1.0 / np.sqrt(self.m))
        return feat




In [7]:
df["lat"] = df["location"].apply(lambda x:eval(x)[0])

In [8]:
df["lon"] = df["location"].apply(lambda x:eval(x)[1])

In [9]:
data = df.to_dict(orient="records")

In [10]:
lat_keys=('lat','latitude'),
lon_keys=('lon','lng','long','longitude','log'),
latlon = []
names = []
D=8
scales_km=(1200,400,150,50)
seed=0

for i, item in enumerate(data):

    lat = item.get("lat") 
    lon = item.get("lon")

    latlon.append([float(lat), float(lon)])
    names.append(item.get('name'))

enc = GeoFourierEncoder(D=D, scales_km=scales_km, seed=seed)
X = enc.transform(np.array(latlon))
   

In [11]:
np.save("geo_B.npy", enc.B)


In [12]:
X.shape

(1162, 8)

In [13]:
len(df)

1162

In [14]:
df["fourier_features"] = list(X)


In [15]:
df["fourier_features"][8]

array([ 0.18806185, -0.48807046, -0.3494173 , -0.24341199,  0.46328473,
        0.10856901, -0.35764167, -0.43675005], dtype=float32)

In [ ]:
import numpy as np
import pandas as pd


X = np.stack(df["fourier_features"].to_numpy())        # (1098, 8)
depts = df["department_name"].to_numpy()


Xn = X / np.linalg.norm(X, axis=1, keepdims=True)
S = Xn @ Xn.T                                          # (1098, 1098)

# Masks
same = depts[:, None] == depts[None, :]                # same department
diag = np.eye(len(df), dtype=bool)

# Per-department stats
rows = []
for d in np.unique(depts):
    idx = np.where(depts == d)[0]
    n = len(idx)

    # Intra (exclude self-sim on diagonal)
    intra_block = S[np.ix_(idx, idx)]
    if n > 1:
        intra_vals = intra_block[~np.eye(n, dtype=bool)]
        intra_mean = float(np.mean(intra_vals))
    else:
        intra_mean = np.nan  # not defined with only 1 row

    # Inter (this dept vs all others)
    other_idx = np.where(depts != d)[0]
    inter_vals = S[np.ix_(idx, other_idx)]
    inter_mean = float(np.mean(inter_vals)) if inter_vals.size else np.nan

    rows.append({"department_name": d, "n_items": n,
                 "intra_mean_cosine": intra_mean,
                 "inter_mean_cosine": inter_mean})

dept_similarity = pd.DataFrame(rows).sort_values("department_name").reset_index(drop=True)
dept_similarity


,department_name,n_items,intra_mean_cosine,inter_mean_cosine
0,Amazonas,12,0.456811,0.413038
1,Antioquia,126,0.668076,0.486846
2,Arauca,8,0.595104,0.445474
3,Atlantico,24,0.964690,0.406156
4,Bogotá D.C.,2,1.000000,0.487766
5,Bolivar,47,0.579751,0.462716
6,Boyaca,124,0.727069,0.462372
7,Caldas,28,0.821388,0.541059
8,Caqueta,17,0.667448,0.484623
9,Casanare,20,0.685947,0.430967


In [ ]:
df.to_json("../files/processed/final_datasets/cities_with_fourier.json")